<h2 align='center'> Stainless Steel Classification </h3>

<p> The General idea of this notebook is to highlight all the process and subprocesses done for Stainless Steel Classification.<br>For this project we have used the dataset provided <a href="https://www.kaggle.com/code/kerneler/starter-tig-stainless-steel-304-ee6df282-0/data">Here</a>. </p>

<h4> <u> Dataset Discussion </u> </h4>
<div>
    <img src='Assets\Dataset_Labels.png' align='right' width=600 height=300>
    <p align='left'><br>Our Dataset has been label encoded as following: <br> The idea is that provided an image our classifier will be able to classify the results as one of the following labels. <br><br>["good weld","burn through","contamination","lack of fusion","lack of shielding gas","high travel speed"]<br><br>
    Since we are following supervised approach we have a json file which maps each image with corresponding output labels which can be used to map each image with corresponding labels. <br><br> The mentioned json file can be seen below: <br> <br></p>
    <img src='Assets\json_files.png' width=550>
</div>


<h3> 0. Intalling Dependencies </h3>

In [36]:
!pip install tensorflow 
!pip install -U scikit-learn
!pip install numpy
!pip install pillow


  Using cached Pillow-9.4.0-cp310-cp310-win_amd64.whl (2.5 MB)


<h3> 1. Importing Dependencies </h3>

Let's import all the dependencies so we can use it later

In [72]:
import json
import numpy as np 
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf

np.set_printoptions(threshold=sys.maxsize)

<h3> 1. PreProcessing </h3>

<p> Once Downloaded please unzip the dataset on the same working directory and the folder directory should be: <br>

<i>../ss304/train/train.json</i>
</p>


In [80]:
def convert_to_array(i,features,f):

    if f==0:
        image=features[i]
        print(f'loading {i+1} out of {len(features)} Images Loaded')
        
    else: 
        image=i
    
    image = Image.open(image)
    image=image.resize((200,200))
    numpydata = np.asarray(image)
    
    return numpydata


def get_images_array_and_labels(path):
    
    with open(path,'r') as datasets:
        datasets=json.load(datasets)
    features=list(datasets.keys())
    features=['/'.join(path.split('\\')[0:-1])+'/'+str(i) for i in features]
    features=np.array([convert_to_array(i,features,0) for i in range(len(features))])
    labels=np.array(list(datasets.values())).reshape(-1,1)
    labels=tf.keras.utils.to_categorical(labels)
    return features,labels
    
    

X,Y=get_images_array_and_labels('ss304\\train\\train.json')

loading 1 out of 24204 Images Loaded
loading 2 out of 24204 Images Loaded
loading 3 out of 24204 Images Loaded
loading 4 out of 24204 Images Loaded
loading 5 out of 24204 Images Loaded
loading 6 out of 24204 Images Loaded
loading 7 out of 24204 Images Loaded
loading 8 out of 24204 Images Loaded
loading 9 out of 24204 Images Loaded
loading 10 out of 24204 Images Loaded
loading 11 out of 24204 Images Loaded
loading 12 out of 24204 Images Loaded
loading 13 out of 24204 Images Loaded
loading 14 out of 24204 Images Loaded
loading 15 out of 24204 Images Loaded
loading 16 out of 24204 Images Loaded
loading 17 out of 24204 Images Loaded
loading 18 out of 24204 Images Loaded
loading 19 out of 24204 Images Loaded
loading 20 out of 24204 Images Loaded
loading 21 out of 24204 Images Loaded
loading 22 out of 24204 Images Loaded
loading 23 out of 24204 Images Loaded
loading 24 out of 24204 Images Loaded
loading 25 out of 24204 Images Loaded
loading 26 out of 24204 Images Loaded
loading 27 out of 242

In [90]:
def create_cnn_model():
    model=tf.keras.models.Sequential() #if imported from tensorflow.keras import models we would write models.Sequential

    #Convlution Process Starts here.
    model.add(tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(200, 200, 1)))
    model.add(tf.keras.layers.MaxPool2D((2,2)))
    model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
    model.add(tf.keras.layers.MaxPool2D((2,2)))
    
    model.add(tf.keras.layers.Conv2D(128,(3,3),activation='relu'))
    model.add(tf.keras.layers.MaxPool2D((2,2)))
    
    model.add(tf.keras.layers.Conv2D(128,(3,3),activation='relu'))
    model.add(tf.keras.layers.MaxPool2D((2,2)))
    
    model.add(tf.keras.layers.Conv2D(128,(3,3),activation='relu'))
    model.add(tf.keras.layers.MaxPool2D((2,2)))
    
    model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
    
    #Fully Connected Layer from here 
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64,activation='relu'))
    model.add(tf.keras.layers.Dense(6,activation='softmax')) 
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    
    #Compile the CNN Model
    model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    model.summary()
    return model
    
CNN_model=create_cnn_model()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 198, 198, 32)      320       
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 99, 99, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 48, 48, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 46, 46, 128)       73856     
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 23, 23, 128)     

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
#print(X_test[1].shape)
history_for_plotting=CNN_model.fit(X_train,y_train,epochs=3,validation_data=(X_test,y_test),shuffle=True)

Epoch 1/3
 79/507 [===>..........................] - ETA: 2:52 - loss: 0.0068 - accuracy: 0.9984

KeyboardInterrupt: 

In [89]:
test_image=convert_to_array('ss304/test/160705-121434-50mmLens added slug/image-0001.png',[],1)
prediction=CNN_model.predict(test_image.reshape(1,200,200,1))
print(np.argmax(prediction))

1/1 [==============================] - 0s 21ms/step
2
